<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Performance Python

Dr. Yves J. Hilpisch

The Python Quants GmbH

<a href='mailto:yves@tpq.io'>yves@tpq.io</a> | <a href='http://tpq.io'>http://tpq.io</a>

When it comes to performance critical applications two things should always be checked: are we using the right _implementation paradigm_ and are we using the right _performance libraries_? This part introduces the following **performance libraries**:

* **numexpr**
* **multiprocessing**
* **Numba**
* **Cython**

A function we use regularly to **compare performance**.

In [ ]:
def perf_comp_data(func_list, data_list, rep=3, number=1):
    ''' Function to compare the performance of different functions.
    
    Parameters
    ==========
    func_list : list
        list with function names as strings
    data_list : list
        list with data set names as strings
    rep : int
        number of repetitions of the whole comparison
    number : int
        number of executions for every function
    '''
    from timeit import repeat
    res_list = {}
    for name in enumerate(func_list):
        stmt = name[1] + '(' + data_list[name[0]] + ')'
        setup = "from __main__ import " + name[1] + ', ' \
                                    + data_list[name[0]]
        results = repeat(stmt=stmt, setup=setup,
                         repeat=rep, number=number)
        res_list[name[1]] = sum(results) / rep
    res_sort = sorted(res_list.iteritems(),
                      key=lambda (k, v): (v, k))
    for item in res_sort:
        rel = item[1] / res_sort[0][1]
        print 'function: ' + item[0] + \
              ', av. time sec: %9.5f, ' % item[1] \
            + 'relative: %6.1f' % rel

## Python Idioms and Performance

In finance, like in other scientific and data-intensive disciplines, numerical computations on large data sets can be quite time consuming. Consider the following **mathematical expression**.

$$
\begin{equation}
y = \sqrt{|\cos(x)|} + \sin(2 + 3x)
\end{equation}
$$

This is easily translated into a **Python function**.

In [ ]:
from math import *
def f(x):
    return abs(cos(x)) ** 0.5 + sin(2 + 3 * x)

Using the `range` function we can generate efficiently a **`list` object with 500,000 numbers** which we can work with.

In [ ]:
I = 500000
a_py = range(I)

Consider now **different implementations** (1).

In [ ]:
def f1(a):
    res = []
    for x in a:
        res.append(f(x))
    return res

Consider now **different implementations** (2).

In [ ]:
def f2(a):
    return [f(x) for x in a]

Consider now **different implementations** (3).

In [ ]:
def f3(a):
    ex = 'abs(cos(x)) ** 0.5 + sin(2 + 3 * x)'
    return [eval(ex) for x in a]

Consider now **different implementations** (4).

In [ ]:
import numpy as np

In [ ]:
a_np = np.arange(I)

In [ ]:
def f4(a):
    return (np.abs(np.cos(a)) ** 0.5 +
            np.sin(2 + 3 * a))

Consider now **different implementations** (5).

In [ ]:
import numexpr as ne

In [ ]:
def f5(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(1)
    return ne.evaluate(ex)

Consider now **different implementations** (6).

In [ ]:
def f6(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(8)
    return ne.evaluate(ex)

In total, the same task, i.e. the evaluation of the numerical expression on an array of size 500,000, is implemented in six different ways:

* standard Python function with explicit looping
* iterator/list comprehension approach with implicit looping
* iterator/list comprehension approach with implicit looping and using `eval`
* `NumPy` vectorized implementation
* single-threaded implementation using `numexpr`
* multi-threaded implementation using `numexpr`

**Exercise**: Check whether all implementations yield the same results.

Let us **compare the performance**.

In [ ]:
func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
data_list = ['a_py', 'a_py', 'a_py', 'a_np', 'a_np', 'a_np']

In [ ]:
perf_comp_data(func_list, data_list, rep=3)

## Memory Layout and Performance

A subtle but sometimes important topic is **memory layout** with `NumPy` arrays.

In [ ]:
import numpy as np
import seaborn as sns; sns.set()
import warnings; warnings.simplefilter('ignore')

In [ ]:
np.zeros((3, 3), dtype=np.float64, order='C')
  # C layout

Consider the `C`-like, i.e. **row-wise**, storage.

In [ ]:
c = np.array([[ 1.,  1.,  1.],
              [ 2.,  2.,  2.],
              [ 3.,  3.,  3.]], order='C')

In this case, the 1s, the 2s and the 3s are stored next to each other.

By contrast, consider the `Fortran`-like, i.e. **column-wise**, storage.

In [ ]:
f = np.array([[ 1.,  1.,  1.],
              [ 2.,  2.,  2.],
              [ 3.,  3.,  3.]], order='F')

Now, the data is stored in a way that 1, 2, 3 are next to each other for each column.

Let's see whether the memory layout makes a difference in some way when the **array is large**.

In [ ]:
x = np.random.standard_normal((3, 1500000))
C = np.array(x, order='C')
F = np.array(x, order='F')
x = 0.0

 First, calculating **sums** with C order.

In [ ]:
%timeit C.sum(axis=0)

In [ ]:
%timeit C.sum(axis=1)

Second, **standard deviations** with C order.

In [ ]:
%timeit C.std(axis=0)

In [ ]:
%timeit C.std(axis=1)

Third, **sums** with F order.

In [ ]:
%timeit F.sum(axis=0)

In [ ]:
%timeit F.sum(axis=1)

Fourth, **standard deviations** with F order.

In [ ]:
%timeit F.std(axis=0)

In [ ]:
%timeit F.std(axis=1)

In [ ]:
C = 0.0; F = 0.0

## Multiprocessing

Sometimes it is, however, helpful to parallelize code execution locally. Here, the `multiprocessing` module of Python might prove beneficial.

In [ ]:
import multiprocessing as mp

Consider the following **function to simulate a geometric Brownian motion**.

In [ ]:
import math
import numpy as np
def simulate_geometric_brownian_motion(p):
    M, I = p
      # time steps, paths
    S0 = 100; r = 0.05; sigma = 0.2; T = 1.0
      # model parameters
    dt = T / M
    paths = np.zeros((M + 1, I))
    paths[0] = S0
    for t in range(1, M + 1):
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt +
                    sigma * math.sqrt(dt) * np.random.standard_normal(I))
    return paths

This function returns **simulated paths given the parametrization for `M` and `I`**.

In [ ]:
paths = simulate_geometric_brownian_motion((5, 2))
paths

Let us implement a **test series** on notebook with 4 cores and the following parameter values. 

In [ ]:
I = 25000  # number of paths
M = 20  # number of time steps
t = 32  # number of tasks/simulations

In [ ]:
# running on machine with 4 cores/threads
from time import time
times = []
for w in range(1, 5):
    t0 = time()
    pool = mp.Pool(processes=w)
      # the pool of workers
    result = pool.map(simulate_geometric_brownian_motion, t * [(M, I), ])
      # the mapping of the function to the list of parameter tuples
    times.append(time() - t0)

**Performance scales linearly** with the number of cores used.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(range(1, 5), times)
plt.plot(range(1, 5), times, 'ro')
plt.grid(True)
plt.xlabel('number of processes')
plt.ylabel('time in seconds')
plt.title('%d Monte Carlo simulations' % t)

## Dynamic Compiling

`Numba` is an open source, `NumPy`-aware optimizing compiler for `Python` code (cf. http://numba.pydata.org). It uses the LLVM compiler infrastructure. Cf. http://www.llvm.org.

### Introductory Example

Let us start with a problem that typically leads to performance issues in Python: **alogrithms with nested loops**.

In [ ]:
from math import cos, log
def f_py(I, J):
    res = 0
    for i in range(I):
        for j in range (J):
            res += 1# int(cos(log(1)))
    return res

In [ ]:
I, J = 5000, 5000
%time f_py(I, J)

In principle, this **can be vectorized** with the help of `NumPy ndarray` objects.

In [ ]:
a = np.ones((I, J), dtype=np.float64)
def f_np(I, J):
    return int(np.sum(np.cos(np.log(a)))), a

In [ ]:
%time res, a = f_np(I, J)

However, the `ndarray` object consumes **200 MB** of memory.

In [ ]:
a.nbytes

Consider therefore the **`Numba` alternative**.

In [ ]:
import numba as nb

In [ ]:
f_nb = nb.jit(f_py)

The new function can be **called directly from Python**. At the first time, with a "huge" overhead.

In [ ]:
%time res = f_nb(I, J)

Let us **compare the performance** of the different alternatives more systematically.

In [ ]:
func_list = ['f_py', 'f_np', 'f_nb']
data_list = 3 * ['I, J']

In [ ]:
perf_comp_data(func_list, data_list, rep=3)

### Binomial Option Pricing

Consider a parametrization for the **binomial option pricing model** of Cox-Ross-Rubinstein (1979) as follows.

In [ ]:
from math import *

In [ ]:
# model & option arameters
S0 = 100.  # initial index level
T = 1.  # call option maturity
r = 0.05  # constant short rate
vola = 0.20  # constant volatility factor of diffusion

# time parameters
M = 1000  # time steps
dt = T / M  # length of time interval
df = exp(-r * dt)  # discount factor per time interval

# binomial parameters
u = exp(vola * sqrt(dt))  # up-movement
d = 1 / u  # down-movement
q = (exp(r * dt) - d) / (u - d)  # martingale probability

An implementation of the **binomial algorithm for European options** consists mainly of these parts:

* **index level simulation**
* **inner value calculation**
* **risk-neutral discounting**

In Python this might take on the form (using `NumPy` arrays).

In [ ]:
import numpy as np
def binomial_py(strike):
    # LOOP 1 - Index Levels
    S = np.zeros((M + 1, M + 1), dtype=np.float64)
      # index level array
    S[0, 0] = S0
    z1 = 0
    for j in xrange(1, M + 1, 1):
        z1 = z1 + 1
        for i in xrange(z1 + 1):
            S[i, j] = S[0, 0] * (u ** j) * (d ** (i * 2))
            
    # LOOP 2 - Inner Values
    iv = np.zeros((M + 1, M + 1), dtype=np.float64)
      # inner value array
    z2 = 0
    for j in xrange(0, M + 1, 1):
        for i in xrange(z2 + 1):
            iv[i, j] = max(S[i, j] - strike, 0)
        z2 = z2 + 1
        
    # LOOP 3 - Valuation
    pv = np.zeros((M + 1, M + 1), dtype=np.float64)
      # present value array
    pv[:, M] = iv[:, M]  # initialize last time point
    z3 = M + 1
    for j in xrange(M - 1, -1, -1):
        z3 = z3 - 1
        for i in xrange(z3):
            pv[i, j] = (q * pv[i, j + 1] +
                        (1 - q) * pv[i + 1, j + 1]) * df
    return pv[0, 0]

This function returns the **present value of a European call option** with parameters as specified before.

In [ ]:
%time round(binomial_py(100), 3)

First improvement: **`NumPy` vectorization**.

In [ ]:
def binomial_np(strike):
    # Index Levels with NumPy
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md
    
    # Valuation Loop
    pv = np.maximum(S - strike, 0) 
    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        pv[0:M - z, t] = (q * pv[0:M - z, t + 1]
                        + (1 - q) * pv[1:M - z + 1, t + 1]) * df
        z += 1
    return pv[0, 0]

**Exercise**: Step through the single construction steps and have a look behind the `NumPy` scene.

Check the **performance of the `NumPy` version**.

In [ ]:
M = 1000  # reset number of time steps
%time round(binomial_np(100), 3)

Let us try the **`Numba` dynamic compiling** approach (remember the overhead of the first call).

In [ ]:
binomial_nb = nb.jit(binomial_py)

In [ ]:
%time round(binomial_nb(100), 3)

A **rigorous performance comparison**.

In [ ]:
func_list = ['binomial_py', 'binomial_np', 'binomial_nb']
K = 100.
data_list = 3 * ['K']

In [ ]:
perf_comp_data(func_list, data_list, rep=3)

In summary, we can state the following:

* **efficiency**: using `Numba` involves only little additional effort
* **speed-up**: `Numba` leads often to significant improvements of execution speed, even compared to vectorized `NumPy` implementations
* **memory**: with `Numba` there is no need to initialize large array objects; the compiler specializes the machine code to the problem at hand and maintains memory efficiency

## Static Compiling with Cython

`Cython` is a static compiler for (annotated) Python code. In effect, `Cython` represents a hybrid language between Python and `C` as the name already suggests.

Again, a simple **example function** with a nested loop.

In [ ]:
def f_py(I, J):
    res = 0.  # we work on a float object
    for i in xrange(I):
        for j in xrange (J * I):
            res += 1
    return res

In [ ]:
I, J = 500, 500
%time f_py(I, J)

Consider now the `Cython` file with the static type declarations (and note the suffic .pyx)

In [ ]:
# uncomment the following line to load the example file
# %load nested_loop.pyx

When no special `C` modules are needed, there is an easy way to **import such a module**, namely via `pyximport`.

In [ ]:
import pyximport
pyximport.install()

This allows us now to **directly import from the `Cython` module**.

In [ ]:
from nested_loop import f_cy

Now, we can check the **performance of the `Cython` function**.

In [ ]:
%time res = f_cy(I, J)

In [ ]:
res

When working in `Jupyter Notebook` there is a **more convenient way** to use `Cython`: `cythonmagic`.

In [ ]:
%load_ext Cython

In [ ]:
%%cython
#
# Nested loop example with Cython
#
def f_cy(int I, int J):
    cdef unsigned int i, j
    cdef double res = 0
    # double float much slower than int or long
    for i in range(I):
        for j in range (J * I):
            res += 1
    return res

In [ ]:
%time res = f_cy(I, J)

In [ ]:
res

Let us return **`Numba`**.

In [ ]:
import numba as nb

In [ ]:
f_nb = nb.jit(f_py)
res = f_nb(I, J)

In [ ]:
%timeit f_nb(I, J)

In [ ]:
res

Finally, again the more **rigorous comparison**.

In [ ]:
func_list = ['f_py', 'f_cy', 'f_nb']
I, J = 500, 500
data_list = 3 * ['I, J']

In [ ]:
perf_comp_data(func_list, data_list, rep=3)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:yves@tpq.io">yves@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="http://hilpisch.com" target="_blank">http://hilpisch.com</a> 

**Quant Platform** &mdash; <a href="http://quant-platform.com" target="_blank">http://quant-platform.com</a>

**Python for Finance** &mdash; <a href="http://python-for-finance.com" target="_blank">http://python-for-finance.com</a>

**Derivatives Analytics with Python** &mdash; <a href="http://derivatives-analytics-with-python.com" target="_blank">http://derivatives-analytics-with-python.com</a>

**Python Trainings** &mdash; <a href="http://training.tpq.io" target="_blank">http://training.tpq.io</a>